# pytorch on CPU performance
 deals with running time of pytorch models on different pc, and performence issues.
 
 1. CPU optimization with MKL/OpenBLAS - config for different CPUs
 2. Number of CPU Threads - how many threads and their effect
 3. CPU thermal issues - temperature issues
 4. Background Load - CPU and memory ussage and their effect - TBD
 5. Storage performance - type of memoty access and disk speed - TBD
 6. Memory Bandwidth/NUMA effects - ?- TBD
 7. python and pytorch version - TBD
 8. model specipic issues - TBD

In [1]:
#!pip install torch
import torch
import os

## CPU optimization with MKL/OpenBLAS - config for different CPUs

* is  pytorch using modern instructions(VX2\AVX512) and not SSE

* does Thermal Throteling prevents good use of heavy instructions?TBD.....................

* does the worksattion is faster for a single(or 2) threads? i can check by setting the thread number

* does the MKL\OpenBlas deafault to using a lot of threads= you could check the env var like  "MKL_NUM_THREADS", they could even if you a single thread loop in pytorch

* does using alot of threads is bad?= can cause too much overhead(simple matrix multiplication runs best on 4 threads and slow on 32, so a cpu with less cores could default to a more afficient number of threads

* env variables which may force bad threading(OMP_NUM_THREADS\MKL_NUM_THREADS\NUMEXPR_MAX_THREADS\MKL_DEBUG_CPU_TYPE), what each does?

### Concepts
 **MKL** -  library for math and matrix multiplication optimization
 
 **OpenMP** - library for multithreading
 
 **NUMA_aware** -
 
 **CPU instructions** - (AVX\AVX2\AVX-512\SSE...), in modern cpu should be at least AVX2
 
 **Threads scheduler** - 
 
 **Number of Threads** - too much threads could do too much overhead
 
 **OpenBLAS** - open source MKL like library probably slower
 
 **Eigen** - 

In [2]:
print(torch.__config__.show())

print("\nenv variables")
print("\t",{k: v for k, v in os.environ.items() if "OMP" in k or "MKL" in k or "NUM" in k})

print(os.environ.get("MKL_NUM_THREADS"))

print("\nnumber of threads:")
print(torch.get_num_threads())

PyTorch built with:
  - C++ Version: 199711
  - MSVC 192829337
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CXX_COMPILER=C:/actions-runner/_work/pytorch/pytorch/builder/windows/tmp_bin/sccache-cl.exe, CXX_FLAGS=/DWIN32 /D_WINDOWS /GR /EHsc /w /bigobj -DUSE_PTHREADPOOL -openmp:experimental -IC:/actions-runner/_work/pytorch/pytorch/builder/windows/mkl/include -DNDEBUG -DUSE_KINETO -DLIBKINETO_NOCUPTI -DUSE_FBGEMM -DUSE_XNNPACK -DSYMBOLICATE_MOBILE_DEBUG_HANDLE -DEDGE_PROFILER_USE_KINETO, LAPACK_INFO=mkl, PERF_WITH_AVX=1, PERF_WITH_AVX2=1, PERF_WITH_AVX512=1, TORCH_VERSION=1.13.1, USE_CUDA=0, USE_CUDNN=OFF, USE_EXCEPTION_PTR=1, USE_GFLAGS=OFF, USE_GLOG=OFF, USE_MKL=ON, USE_MKLDNN=ON, USE_MPI=OF

in this case :

  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CXX_COMPILER=C:/actions-runner/_work/pytorch/pytorch/builder/windows/tmp_bin/sccache-cl.exe, CXX_FLAGS=/DWIN32 /D_WINDOWS /GR /EHsc /w /bigobj -DUSE_PTHREADPOOL -openmp:experimental -IC:/actions-runner/_work/pytorch/pytorch/builder/windows/mkl/include -DNDEBUG -DUSE_KINETO -DLIBKINETO_NOCUPTI -DUSE_FBGEMM -DUSE_XNNPACK -DSYMBOLICATE_MOBILE_DEBUG_HANDLE -DEDGE_PROFILER_USE_KINETO, LAPACK_INFO=mkl, PERF_WITH_AVX=1, PERF_WITH_AVX2=1, PERF_WITH_AVX512=1, TORCH_VERSION=1.13.1, USE_CUDA=0, USE_CUDNN=OFF, USE_EXCEPTION_PTR=1, USE_GFLAGS=OFF, USE_GLOG=OFF, USE_MKL=ON, USE_MKLDNN=ON, USE_MPI=OFF, USE_NCCL=OFF, USE_NNPACK=OFF, USE_OPENMP=ON, USE_ROCM=OFF, 
  
 * your using MKL version 2020.0.2(for linear algebra)
 
 * and MKL-DNN v2.6.0(for deep learning apps)
 * and OPENMP(for multithreading)
 * and LAPACK(for matrix solvers)
 
 * CPU uses VX2
 
 * BLAS using MKL
 * both MKL and MKL-DNN and OpenMP are on and enabled
 * CUDA and CUDNN are off and disabled
 
 * all AVX(2\512) are enablesd and the use is on AVX2 because the cpu probably dont support AVX512
 
 

### to control number of threads

In [3]:
print(os.environ.get("MKL_NUM_THREADS"))
#os.environ["MKL_NUM_THREADS"]=4
# and
#torch.set_num_threads(4)
# and
#os.environ["OMP_NUM_THREADS"]=4
# now the script is using 4 threads


None


## CPU thermal issues
the speed under full load should be around the base clock value if its below there is some throtelling, you can tell clocks values by the cpu model 
### concepts

**Base clock** - minimum guretedd speed under load without over heating
**Boost clock** - higher clock can be reach temprarly, ussually when the load starts it boost and then goes to base speed